In [ ]:
from classModule.BasicClass import *
from classModule.ClassFunc import * 
from classModule.sprint1Func import *
import os
import time
import datetime
from datetime import date
import prettytable as pt

valid_tags = {0:["INDI","FAM","HEAD","TRLR","NOTE"],\
            1:["NAME","SEX","BIRT","DEAT","FAMC","FAMS","MARR","HUSB","WIFE","CHIL","DIV"]\
         ,2:"DATE"}

def monthToNum(shortMonth):
    return{
            'JAN' : 1,
            'FEB' : 2,
            'MAR' : 3,
            'APR' : 4,
            'MAY' : 5,
            'JUN' : 6,
            'JUL' : 7,
            'AUG' : 8,
            'SEP' : 9, 
            'OCT' : 10,
            'NOV' : 11,
            'DEC' : 12
    }[shortMonth]

def readAndSaveToList(filePath, List, List2):
    
    #read gedcom file
    print(filePath)
    input_ged = open(filePath, "r") 
   # saveFile="output_"+filename+".txt"
   # writeFile = open(saveFile, "w")
    #set flags
    flag_Birth=False
    flag_Death=False
    flag_Married=False
    flag_Divorced=False
    #read each line to input

    for each_input in input_ged:
        # Ensure line is not empty or comments
        if len(each_input)>0 and each_input[:2]!="--":
            each_input=each_input.rstrip("\n")
          #  print(each_input)
            each_input_split=each_input.split()
            # Ensure there're level and tag
            if len(each_input_split)>=2:
                level,tag=int(each_input_split[0]),each_input_split[1]
                # Ensure input is valid
                word_count=2+len(tag)+1
                # Ensure there's desc to process
                if len(each_input_split)>=3:
                    desc=each_input_split[2]
                    # Detect New Individual and create new class
                    if desc=="INDI":
                        ID=tag[0:3]
                        p1=Individuals(ID)
                        List.append(p1)
                    # Detect New Family and create new class
                    if desc=="FAM":
                        ID=tag
                        f1=Families(ID)
                        List2.append(f1)    
                    if tag=="SEX":
                        SEX=desc  
                        p1.Gender=SEX    
                    if tag=="NAME":
                        NAME=each_input[word_count:].strip()
                        p1.Name=NAME
                    if flag_Birth==True and tag=="DATE":
                        B_DAY=int(desc)
                        B_MONTH=int(monthToNum(each_input_split[3]))
                        B_YEAR=int(each_input_split[4])
                        
                        Birthday=datetime.date(B_YEAR,B_MONTH,B_DAY)
                        p1.Age=int ((date.today()-Birthday)/datetime.timedelta (days=1)/365)
                        p1.Birthday=str(Birthday)
                    
                        flag_Birth=False   
                    if flag_Death==True and tag=="DATE":
                        D_DAY=int(desc)
                        D_MONTH=int(monthToNum(each_input_split[3]))
                        D_YEAR=int(each_input_split[4])
                        
                        Death=datetime.date(D_YEAR,D_MONTH,D_DAY)
                        p1.Age=int((Death-Birthday)/datetime.timedelta (days=1)/365)
                        p1.Death=str(Death)

                        flag_Death=False
                    if flag_Married==True and tag=="DATE":
                        M_DAY=int(desc)
                        M_MONTH=int(monthToNum(each_input_split[3]))
                        M_YEAR=int(each_input_split[4]) 
                        MarriedDay=datetime.date(M_YEAR,M_MONTH,M_DAY)
                        f1.Married=str(MarriedDay)
                        
                        flag_Married=False
                    if flag_Divorced==True and tag=="DATE":
                        Div_DAY=int(desc)
                        Div_MONTH=int(monthToNum(each_input_split[3]))
                        Div_YEAR=int(each_input_split[4]) 
                        DivorcedDay=datetime.date(Div_YEAR,Div_MONTH,Div_DAY)
                        f1.Divorced=str(DivorcedDay)
                        
                        flag_Divorced=False
                    if tag=="FAMS":
                        Spouse=desc
                        p1.Spouse=Spouse
                    if tag=="FAMC":
                        Child=desc
                        p1.Child=Child
                    if tag=="HUSB":
                        Husband=desc
                        f1.HusbandID=Husband
                        for each in List:
                            if each.ID==f1.HusbandID: 
                                f1.HusbandName=each.Name      
                    if tag=="WIFE":
                        Wife=desc
                        f1.WifeID=Wife
                        for each in List:
                            if each.ID==f1.WifeID: 
                                f1.WifeName=each.Name
                    if tag=="CHIL":
                        Children=desc
                        f1.Children.append(Children)
                elif len(each_input_split)>=2:
                    if tag=="BIRT":
                        flag_Birth=True
                        p1.Alive="True"
                    if tag=="DEAT":
                        flag_Death=True
                        p1.Alive="False"
                    if tag=="MARR":
                        flag_Married=True
                    if tag=="DIV":
                        flag_Divorced=True
                        

In [ ]:
# input sample
file_name = 'UserStory.ged'
filePath = os.path.join(os.getcwd(), "Sample/" + file_name)

# create list for individual and families
individualList=[]
familyList=[]

# read file to create individual and families
if os.path.exists(filePath):
    readAndSaveToList(filePath, individualList, familyList)
else:
    print("File doesn't exist")

In [ ]:
outputFile="outputFile.txt"

writefile = open(outputFile,"w") 
 


In [ ]:
#Making PrettyTable

tb=pt.PrettyTable()
tb.field_names=["ID","Name","Gender","Birthday","Age","Alive","Death"\
                ,"Child","Spouse"]
for each in individualList:
    tb.add_row([each.ID,each.Name,each.Gender,each.Birthday,each.Age,\
              each.Alive,each.Death,each.Child,each.Spouse])
print(tb)
writefile.write("Individuals\n") 
writefile.write(tb.get_string()) 
writefile.write("\n") 

tb2=pt.PrettyTable()
tb2.field_names=["ID","Married","Divorced","Husband ID","Husband Name","Wife ID","Wife Name"\
                ,"Children"]
for each in familyList:
    tb2.add_row([each.ID,each.Married,each.Divorced,each.HusbandID,each.HusbandName,\
              each.WifeID,each.WifeName,each.Children])
print(tb2)
writefile.write("Families\n") 
writefile.write(tb2.get_string()) 

In [ ]:
    
# # checking all family (User Story 05 Marriage before death)
# for fm in familyList:    
#     currHusband = getItemByID(individualList, fm.HusbandID)
#     currWife = getItemByID(individualList, fm.WifeID)
#     isDateBeforeCur(fm.Married)
#     isDateBeforeCur(fm.Divorced)
#     isMarriageBeforeDeath(fm.Married, currHusband.Death)
#     isMarriageBeforeDeath(fm.Married, currWife.Death)
#     #isValidDivorce(fm.Divorced,currHusband.Death)
#     #isValidDivorce(fm.Divorced,currWife.Death)

In [ ]:
from classModule.sprint1Func import *

writefile.write("\n")
# checking all individual (User Story 03 Birth before death)
for pp in individualList:
    if(isDateBeforeCur(pp.Birthday)) != True:
        print("ERROR: INDIVIDUAL: US01: Date: {0} is not before current time.".format(pp.Birthday))
        writefile.write("ERROR: INDIVIDUAL: US01: Date: {0} is not before current time.".format(pp.Birthday)) 
    if(isDateBeforeCur(pp.Death)) != True:
        print("ERROR: INDIVIDUAL: US01: Date: {0} is not before current time.".format(pp.Death))
        writefile.write("ERROR: INDIVIDUAL: US01: Date: {0} is not before current time.".format(pp.Death))     
    ageLess150(pp)
    if(isBirthBeforeDeath(pp.Birthday, pp.Death)) != True:
        print("ERROR: INDIVIDUAL: US03:{0}: {1} Birth before death {2}".format(pp.ID, pp.Birthday, pp.Death))
        writefile.write("ERROR: INDIVIDUAL: US03:{0}: {1} Birth before death {2}\n".format(pp.ID, pp.Birthday, pp.Death)) 
# checking all family (User Story 05 Marriage before death)
for fm in familyList:    
    currHusband = getItemByID(individualList, fm.HusbandID)
    currWife = getItemByID(individualList, fm.WifeID)
    if(isMarriageBeforeDeath(fm.Married, currHusband.Death)) != True:
        print("ERROR: FAMILY: US05:{0}: {1} Marriage before death {2}".format(currHusband.ID, fm.Married, currHusband.Death))
        writefile.write("ERROR: FAMILY: US05:{0}: {1} Marriage before death {2}\n".format(currHusband.ID, fm.Married, currHusband.Death))
    if(isMarriageBeforeDeath(fm.Married, currWife.Death)) != True:
        print("ERROR: FAMILY: US05:{0}: {2} Marriage before death {2}".format(currWife.ID, fm.Married, currWife.Death))
        writefile.write("ERROR: FAMILY: US05:{0}: {2} Marriage before death {2}\n".format(currWife.ID, fm.Married, currWife.Death))
    if(isDateBeforeCur(fm.Married)) != True:
        print("ERROR: INDIVIDUAL: US01: Date: {0} is not before current time.".format(fm.Married))
        writefile.write("ERROR: INDIVIDUAL: US01: Date: {0} is not before current time.".format(fm.Married)) 
    if(isDateBeforeCur(fm.Divorced)) != True:
        print("ERROR: INDIVIDUAL: US01: Date: {0} is not before current time.".format(fm.Divorced))
        writefile.write("ERROR: INDIVIDUAL: US01: Date: {0} is not before current time.".format(fm.Divorced))       
    if(isDivorceBeforeDeath(fm.Divorced,currHusband.Death)) != True:
        print("ERROR: FAMILY: US06:{0}: {1} Divorce after death {2}".format(currHusband.ID, fm.Divorced, currHusband.Death))
        writefile.write("ERROR: FAMILY: US06:{0}: {1} Divorce after death {2}\n".format(currHusband.ID, fm.Divorced, currHusband.Death))
    if(isDivorceBeforeDeath(fm.Divorced,currWife.Death)) != True:
        print("ERROR: FAMILY: US06:{0}: {1} Divorce after death {2}".format(currWife.ID, fm.Divorced, currWife.Death))
        writefile.write("ERROR: FAMILY: US06:{0}: {1} Divorce after death {2}\n".format(currWife.ID, fm.Divorced, currWife.Death))
    birthBFmarried(fm.ID, fm.Married, currHusband.Birthday)
    birthBFmarried(fm.ID, fm.Married, currWife.Birthday)
    US04(fm.ID,fm.Married,fm.Divorced)
    US08(individualList,fm.ID,fm.Children,fm.Married,fm.Divorced)
    
writefile.close()

In [ ]:
US10("2018-05-20","2000-05-20")

In [ ]:
for fm in familyList: 
    if(US10(fm.Married, currHusband.Birthday)) != True:
        print("ERROR: FAMILY: US10:{1}: Married on {0} before he was 14".format(fm.Married,currHusband.ID))
        writefile.write("ERROR: FAMILY: US10:{1}: Married on {0} before he was 14".format(fm.Married,currHusband.ID))
    if(US10(fm.Married, currWife.Birthday)) != True:
        print("ERROR: FAMILY: US10:{1}: Married on {0} before she was 14".format(fm.Married,currWife.ID))
        writefile.write("ERROR: FAMILY: US10:{1}: Married on {0} before she was 14".format(fm.Married,currWife.ID))
    if(US15(fm.Children)) != True:
        print("ERROR: FAMILY: US15:{0}: Siblings not fewer than 15.".format(fm.ID))
        writefile.write("ERROR: FAMILY: US15:{0}: Siblings not fewer than 15.".format(fm.ID))